In [1]:
import huggingface_hub

huggingface_hub.login()

In [12]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)

from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator

In [13]:
from datasets import load_dataset
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split

# Load the dataset from the parquet file
dataset = pl.read_parquet('hf://datasets/wikimedia/wikipedia/20231101.ko/train-00000-of-00003.parquet')

# Convert the Polars DataFrame to a Pandas DataFrame
df = pd.DataFrame(dataset)

# Split the data into training and validation sets with an 80:20 ratio
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the Pandas DataFrames back to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Print the training and validation DataFrames
print("Train DataFrame:")
print(train_dataset)

print("\nValidation DataFrame:")
print(val_dataset)

Train DataFrame:
Dataset({
    features: ['0', '1', '2', '3', '__index_level_0__'],
    num_rows: 172772
})

Validation DataFrame:
Dataset({
    features: ['0', '1', '2', '3', '__index_level_0__'],
    num_rows: 43194
})


In [7]:
train_dataset[:10]

{'0': ['189407',
  '363840',
  '819321',
  '54149',
  '492496',
  '651045',
  '240033',
  '829429',
  '612374',
  '397989'],
 '1': ['https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%83%A0',
  'https://ko.wikipedia.org/wiki/%EC%82%AC%ED%82%A4%EC%9B%90%EC%88%AD%EC%9D%B4',
  'https://ko.wikipedia.org/wiki/%ED%8F%B4%20%ED%86%A0%EB%A8%B8%EC%8A%A4',
  'https://ko.wikipedia.org/wiki/%EB%85%B8%EB%8D%95%EC%88%A0',
  'https://ko.wikipedia.org/wiki/%EC%BF%A8%EB%B7%B0%ED%8B%B0',
  'https://ko.wikipedia.org/wiki/%EC%B5%9C%EA%B1%B4%EC%9A%B0',
  'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EB%8F%84%20%EC%A0%9C21%ED%98%B8%EC%84%A0%20%28%EC%95%88%EB%82%B4%EC%9A%A9%29',
  'https://ko.wikipedia.org/wiki/%EC%98%88%EC%88%98%EC%99%80%20%EA%B0%84%EC%9D%8C%ED%95%9C%20%EC%97%AC%EC%9D%B8',
  'https://ko.wikipedia.org/wiki/%EA%B2%BD%ED%83%9C%20%28%EC%97%B0%ED%98%B8%29',
  'https://ko.wikipedia.org/wiki/FC%20%EB%8B%A4%ED%82%A4%EC%95%84%20%ED%82%A4%EC%8B%9C%EB%84%88%EC%9A%B0'],
 '2': ['스탠

In [9]:
# 필요한 라이브러리 불러오기
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
import torch

# torch_dtype 설정
torch_dtype = torch.float16  # 예시로 torch.float16 사용. 필요에 따라 변경 가능

In [11]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)


model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
              "google/gemma-2-2b-it",
              trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [44]:
from peft import LoraConfig

peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [45]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # epoch는 1로 설정
    max_steps=10000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    warmup_steps=150,  # warmup_steps을 절대값으로 설정
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='tensorboard',
)

In [49]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_params,
    train_dataset=train_dataset,  # Use the Hugging Face Dataset here
    eval_dataset=val_dataset,      # Use the Hugging Face Dataset here
    peft_config=peft_params,
    dataset_text_field="question",  # Specify the appropriate field name
    # Additional parameters as needed
)

trainer.train()

/home/kar/Projects/KoGemma/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/kar/Projects/KoGemma/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/kar/Projects/KoGemma/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/172772 [00:00<?, ? examples/s]

KeyError: 'question'

In [ ]:
# 저장된 체크포인트 경로
checkpoint_path = "./results/checkpoint-10000"

# 모델 로드
model = YourModelClass.from_pretrained(checkpoint_path)

# SFTTrainer 설정 (동일하게 유지)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_params,
    dataset_text_field="question",
    # 필요한 추가적인 파라미터들
)

# 학습 재개
trainer.train(resume_from_checkpoint=checkpoint_path)


In [ ]:
trainer.save_model('./models/20240704')